### Enter full names of group members:

##### Name A:
##### Name B:

In [1]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [2]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500 

In [4]:
def dgim_algorithm(stream_path, N):
    
    # Create the buckets and initialize the timestamp
    pos=1
    bucket_list=[[]]


    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        while True:
            bit = f.read(1)
            
            # Clause to break while loop at the end of the stream
            if not bit:
                break
            
            if bit=="1":
                bucket_list[0].append(pos)
                for b_index, bucket in enumerate(bucket_list):
                    if len(bucket)==3:
                        if len(bucket_list)>b_index+1:
                            bucket_list[b_index+1].append(bucket[1])
                            bucket_list[b_index]=[bucket[2]]
                        else:
                            bucket_list.append([bucket[1]])
                            bucket_list[b_index]=[bucket[2]]
                    else:
                        break
            if len(bucket_list[0])>4:
                break

            pos+=1

    end_time_stamp=pos
            
                            
    return bucket_list, end_time_stamp

In [5]:
bucket = dgim_algorithm(stream_path, N)

In [6]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [[1, 2, 7, 9, 10, 15, 17, 22, 25, 28, 29, 31, 35, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 56, 57, 58, 59, 60, 61, 62, 64, 65, 69, 75, 77, 78, 79, 80, 84, 86, 87, 89, 93, 96, 97, 99, 100, 103, 104, 106, 109, 111, 112, 113, 114, 115, 116, 118, 119, 121, 123, 124, 125, 128, 129, 132, 133, 134, 135, 136, 137, 139, 143, 144, 146, 148, 150, 152, 155, 156, 157, 159, 160, 166, 168, 171, 177, 180, 181, 182, 183, 184, 187, 188, 191, 194, 195, 197, 198, 199, 203, 206, 207, 211, 212, 213, 214, 215, 216, 221, 222, 224, 225, 226, 229, 231, 233, 235, 236, 238, 239, 241, 243, 244, 246, 247, 253, 254, 255, 259, 262, 263, 265, 271, 272, 275, 276, 277, 278, 283, 287, 289, 290, 291, 296, 301, 302, 303, 304, 306, 314, 315, 316, 317, 319, 322, 326, 328, 334, 336, 338, 340, 341, 342, 344, 345, 347, 348, 349, 350, 353, 354, 358, 361, 368, 369, 372, 376, 378, 382, 383, 387, 389, 390, 391, 394, 395, 396, 397, 399, 401, 404, 406, 408, 

#### 1.2. Query the Bucket 

In [6]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [43]:
def dgim_query(bucket, N, k):  
    
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket
   
    
    # To-do! initialize the different variables
    

    # To-do! query the dgim bucket using the k parameters
    
    
    return math.ceil(one_count)

In [8]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [9]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 500, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [10]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [11]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [12]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [13]:
def generate_hash(h, N):
    hash_list = []
    
    # To-do! generate a list of hash functions
        
    return hash_list

In [14]:
hashes = generate_hash(h, bloom_size)

In [15]:
def create_bloom_filter(B, hashes, data):
    with data.open() as f:
        for name in f:
            
            # To-do! update the hash index of the bloom filter with 1s
            
    return B

In [16]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [17]:
bloom_array

array([1., 1., 0., ..., 0., 1., 0.])

#### 2.2. Verify usernames

In [18]:
def single_verify_username(bloom_array, hashes, new_user):
    
    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
        
    return code
    

In [19]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"

In [20]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [21]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username KazeemTDT4305 is available. Congrats!


In [22]:
def group_verify_username(bloom_array, hashes, data):
    # Initialize counts
    total_name = 0
    taken_name = 0
    
    with data.open() as f:
        for name in f:
            # To-do! similar to the single verify, but returns a percentage of usernames taken...
            # ...(In other words seen already by the bloom filter during its creation)
            
    return round(taken_name/total_name*100,2)   

In [23]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.71%
----------------------------------------------------------


### 3. Flajolet-Martin

In [24]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # To-do! Define hash function h(x) = 6x + 1 mod 5
    

    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate

In [25]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [26]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [27]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [28]:
def greedy_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using greedy algorithm
    
    return revenue

In [29]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 8
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 7
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 8
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 8
Trial 10 - Revenue generated: 8
------------------------------------------------
Average revenue generated for all trials:  8.4


#### 4.2. Balance Algorithm

In [30]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0
    
    # To-do! update revenue using balance algorithm
    
    return revenue

In [31]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 9
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 10
Trial 8 - Revenue generated: 9
Trial 9 - Revenue generated: 9
Trial 10 - Revenue generated: 10
-------------------------------------------
Average revenue generated for all trials:  9.2


### 5. Recommender System

In [32]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [33]:
def user_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a user-user CF using cosine similarity as distance measure
    
    return prediction   

In [34]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [35]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.42 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.49 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [36]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure
    
    return prediction

In [37]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

In [38]:
# Enter answer here

#### Case 2

In [39]:
# Enter answer here

#### Case 3

In [40]:
# Enter answer here

#### Case 4

In [41]:
# Enter answer here

#### Case 5

In [42]:
# Enter answer here